In [1]:
import sys
sys.path.append('..')
sys.path.append('../classes/')
sys.path.append('../data/')
sys.path.append('../model_weights/')

from classes.ClockDataset import *
from classes.MinutesDistance import *
from classes.Models import *
from utilities import *
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
weights_path = '../model_weights/periodic'
batch_size=64

<h4>In this notebook we evaluate the pretrained periodic labelled approach on the whole dataset in order to calculate the 'common sense error' (true distance between predictions and targets)</h4>

In [2]:
# load model and loss function
model = NN_regression_2(  input_channels=1,
                        h=150,w=150,
                        n_outputs=4).to(device)
model.load_state_dict(torch.load(weights_path))
loss = nn.MSELoss()

In [3]:
# load data and create dataset
data, labels = load_data('../data')
labels = transform_labels(labels)
clock_dataset = ClockDataset(data, labels, transform=False)
data_loader = DataLoader(clock_dataset, batch_size=batch_size)

In [4]:
# make predictions
approach = "periodic_labels"
predictions = predict(data_loader, model, loss, device, approach)
#true_preds = denormalize_time(predictions)
print(predictions.shape)
# define common sense error
mins_dist = MinutesDistance()
mins_dist_error = mins_dist.minutes_loss(torch.FloatTensor(predictions),torch.FloatTensor(labels[:,:2]))

print(f"Mean error in minutes of predictions for the whole dataset: {mins_dist_error}")

Avg evaluation loss of best weights: 0.007614 

torch.Size([18000, 2])
Mean error in minutes of predictions for the whole dataset: 8.205389022827148
